In [1]:
import pandas as pd
import nltk as nk
import numpy as np

In [3]:
#metro_lyrics.head()

In [4]:
metro_lyrics = pd.read_csv("lyrics.csv", index_col = 'Unnamed: 0')
metro_lyrics.columns = map(str.lower, metro_lyrics.columns)
song_lyrics = pd.read_csv("Song_lyrics.csv", index_col = 'Unnamed: 0')
#del (metro_lyrics['year'])
del (song_lyrics['album'])
whole_data = metro_lyrics.append(song_lyrics, sort = True)

In [5]:
whole_data.shape

(2755, 4)

### Removing genres which have a low number of instances

In [7]:
data = whole_data.drop(whole_data[whole_data.genre=='Other'].index)
data = data.drop(data[data.genre=='Not Available'].index)
data = data.drop(data[data.genre=='Jazz'].index)
data = data.drop(data[data.genre=='Folk'].index)
data = data.drop(data[data.genre=='R&B'].index)
data = data.drop(data[data.genre=='Electronic'].index)
data = data.drop(data[data.genre=='Country'].index)
data = data.drop(data[data.genre=='Indie'].index)

In [8]:
data.groupby('genre').count()

,artist,lyrics,song
genre,,,
Hip-Hop,1614,1614,1613
Pop,98,98,98
Rock,214,214,214


In [31]:
data.shape

(1925, 4)

### Remove stopwords and Stem text

In [32]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
#with open('/Users/purvank/Desktop/NLP IIT/Ipython_Notebooks/stopwords.txt') as f:
#    lines = f.read().splitlines()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shradhitsubudhi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
def StopStem(df):
    #morestop = set(lines)
    stop = set(stopwords.words('english'))
    df['lyrics'] = df['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    #df['lyrics'] = df['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (morestop)]))
    df["lyrics"] = df['lyrics'].str.replace('[^\w\s]','')
    return df

In [34]:
songs = StopStem(data)

In [35]:
songs.head()

,artist,genre,lyrics,song
490,Little Anthony & The Imperials,Rock,Oooh oooo Id move mountain If want Though hard...,So Much
491,Little Anthony & The Imperials,Rock,Well Im alright alright Im alright yes Im alri...,I'm Alright
492,Little Anthony & The Imperials,Rock,You remember But I remember Twas long ago You ...,Tears on My Pillow
493,Little Anthony & The Imperials,Rock,Two People In World Little Anthony Imperials W...,Two People in the World
494,Little Anthony & The Imperials,Rock,Wishful wishful wishful wishful You believe Bu...,Wishful Thinking


### Fix Unicode issues in text :

In [36]:
import ftfy
def fixUni(df):
    df['lyrics'] = df['lyrics'].apply(lambda x: ftfy.fix_text_encoding(x))
    #df['lyrics'] = df['lyrics'].apply(lambda x: x if ftfy.badness.sequence_weirdness(x)>0)
    return df

In [37]:
songs = fixUni(data)

In [38]:
songs.shape

(1925, 4)

In [39]:
songs.shape

(1925, 4)

### Removing instances which have a non-english words in lyrics

In [40]:
for index,row in songs.lyrics.iteritems():
    if(ftfy.badness.sequence_weirdness(row)<0):
        songs.lyrics[index] = np.NaN

In [41]:
songs.dropna(inplace=True)

In [42]:
songs.shape

(1925, 4)

### Removing parenthesis and brackets and removing unimportant words using regular expression.

In [43]:
def remove_brackets(text):
    new_text = re.sub(r'\(.*?\)','',text)
    new_text = re.sub(r'\[.*?\]','',text)
    return new_text

In [44]:
import re
def ReBr(df):
        df['lyrics'] = df['lyrics'].apply(remove_brackets)
        return df

In [45]:
def Chorus(df):
    words = ['Chorus','Verse1','Verse2','Verse']
    df['lyrics'] = df['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in words]))
    return df

In [46]:
songs = ReBr(songs)

In [47]:
songs = Chorus(songs)

In [30]:
songs.to_csv("ReallyReallyCleaned.csv")